In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import sys, copy, emcee, corner

%matplotlib inline
%load_ext autoreload

sys.path.append('./ligbind/')
import ligbind as lb

In [2]:
WT_08_lbl, WT_08_rtot, WT_08_lig, WT_08_sat = lb.datprep('Data/EGFR_WT_08_Sat.csv','Data/EGFR_WT_08_Rtot.csv')
WT_09_lbl, WT_09_rtot, WT_09_lig, WT_09_sat = lb.datprep('Data/EGFR_WT_09_Sat.csv','Data/EGFR_WT_09_Rtot.csv')
WT_11_lbl, WT_11_rtot, WT_11_lig, WT_11_sat = lb.datprep('Data/EGFR_WT_11_Sat.csv','Data/EGFR_WT_11_Rtot.csv')
KI_08_lbl, KI_08_rtot, KI_08_lig, KI_08_sat = lb.datprep('Data/EGFR_KI_08_Sat.csv','Data/EGFR_KI_08_Rtot.csv')
KI_09_lbl, KI_09_rtot, KI_09_lig, KI_09_sat = lb.datprep('Data/EGFR_KI_09_Sat.csv','Data/EGFR_KI_09_Rtot.csv')
KI_09c_lbl, KI_09c_rtot, KI_09c_lig, KI_09c_sat = lb.datprep('Data/EGFR_KI_09c_Sat.csv','Data/EGFR_KI_09c_Rtot.csv')

hootie hoo! the indices match!
hootie hoo! the indices match!
hootie hoo! the indices match!
hootie hoo! the indices match!
hootie hoo! the indices match!
hootie hoo! the indices match!


In [3]:
WT_08_parm = np.array([4.6,5.3,0.34,530.])
WT_09_parm = np.array([4.2,2.0,0.19,860.])
WT_11_parm = np.array([3.4,1.1,0.15,240.])
KI_08_parm = np.array([0.36,48.,0.79,0.61])
KI_09_parm = np.array([0.17,61.,1.9,1.0])
KI_09c_parm = np.array([0.15,75.,0.56,3.7])

In [4]:
WT_08_estr0 = np.array([5.70E-04,1.02E-03,2.18E-03,2.85E-03,5.48E-03,1.06E-02])
WT_09_estr0 = np.array([4.75E-04,1.09E-03,4.44E-03,6.07E-03,2.85E-02])
WT_11_estr0 = np.array([2.85E-04,1.95E-03,2.54E-03,9.18E-03,1.19E-02,3.80E-02])
KI_08_estr0 = np.array([9.73E-04,2.85E-03,4.11E-03,7.00E-03,9.09E-03,1.57E-02])
KI_09_estr0 = np.array([5.22E-04,1.35E-03,2.44E-03,3.56E-03,8.57E-03,1.15E-02])
KI_09c_estr0 = np.array([3.32E-04,9.97E-04,1.54E-03,2.66E-03,4.75E-03,6.24E-03])

In [5]:
#vary all Wym model parameters
#variance weighted by y-value (not y squared)
def lnlik4p(parms,rtot,data,ligs):
    datac = np.concatenate(data)
    modparms = parms[0:-1]
    rtots = rtot
    f = parms[-1]
    model = lb.models.wymfunc(modparms,ligs,rtots)
    invsig2 = 1.0/np.square(f*datac) #do I need to change this to DATAC?
    return -0.5*(np.sum((datac-model)**2*invsig2 - np.log(invsig2)))

#uniform prior based on bounds
def lnpri4p(allfit,bounds):
    assert len(bounds) == len(allfit)
    if all([bounds[i][0] <= allfit[i] <= bounds[i][1] for i in range(len(bounds))]) is True:
        return 0.0
    else:
        return -np.inf
    
def lnprob4p(allfit,bounds,rtot,data,ligs):
    lp = lnpri4p(allfit,bounds)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlik4p(allfit,rtot,data,ligs)

In [6]:
nll = lambda *args: -lnlik4p(*args)

In [7]:
g1 = np.array([4.,6.,13.,170.,0.01])
g2 = np.array([1.,4.,0.5,100.,0.01])
g3 = np.array([15.,6.,0.8,245.,0.01])

In [8]:
bnds = ((0.0001,1000.),(0.0001,1000.),(0.0001,1000.),(0.001,10000.),(1e-6,0.5))

In [9]:
satslist = [WT_08_sat,WT_09_sat,WT_11_sat,KI_08_sat,KI_09_sat,KI_09c_sat]
ligslist = [WT_08_lig,WT_09_lig,WT_11_lig,KI_08_lig,KI_09_lig,KI_09c_lig]
parmslist = [WT_08_parm,WT_09_parm,WT_11_parm,KI_08_parm,KI_09_parm,KI_09c_parm]
rtotslist = [WT_08_rtot,WT_09_rtot,WT_11_rtot,KI_08_rtot,KI_09_rtot,KI_09c_rtot]

In [11]:
def lnlik4pNW(parms,rtot,data,ligs):
    datac = np.concatenate(data)
    modparms = parms[0:-1]
    rtots = rtot
    f = parms[-1]
    model = lb.models.wymfunc(modparms,ligs,rtots)
    #invsig2 = 1.0/np.square(f*datac) #do I need to change this to DATAC?
    #return -0.5*(np.sum((datac-model)**2*invsig2 - np.log(invsig2)))
    return -0.5*(np.sum((datac-model)**2))

In [12]:
nllNW = lambda *args: -lnlik4pNW(*args)

In [13]:
def lsfit(parms,rtots,data,ligs):
    datac = np.concatenate(data)
    calc = lb.models.wymfunc(parms,ligs,rtots)
    return (calc-datac)

In [14]:
lsfit(g1[:-1],rtotslist[0],satslist[0],ligslist[0])

array([-0.01605959, -0.01647449,  0.00478628,  0.02859518,  0.07981032,
        0.08981702,  0.14783095,  0.20544371,  0.10500826,  0.06743183,
        0.0745892 ,  0.0585378 ,  0.06786433,  0.03710495, -0.00473132,
        0.0008296 ,  0.00213457,  0.02672183,  0.071936  ,  0.11298388,
        0.13050943,  0.16625852,  0.16763117,  0.17926722,  0.13025385,
        0.07710727,  0.05921344,  0.02194106, -0.00505427,  0.06627452,
        0.11110174,  0.18528551,  0.21001843,  0.22954627,  0.20454318,
        0.12355516,  0.08335999,  0.05262112,  0.06882666,  0.01591824,
        0.02212876,  0.0070621 ,  0.05664827,  0.09364638,  0.15547705,
        0.19542062,  0.20664856,  0.16524135,  0.14611846,  0.08181202,
        0.11734197,  0.06014753,  0.02814457,  0.04864801,  0.02517652,
        0.1048128 ,  0.19066165,  0.22330683,  0.28950366,  0.32344494,
        0.29396588,  0.22871168,  0.19844775,  0.16924944,  0.06120878,
        0.06509165, -0.0091137 ,  0.01046367,  0.02792933,  0.10

In [40]:
lsbnds = (tuple(1e-3 for i in range(4)),tuple(1000. for i in range(4)))

In [16]:
lsbnds

((0.001, 0.001, 0.001, 0.001), (1000.0, 1000.0, 1000.0, 1000.0))

In [94]:
sp.optimize.least_squares(lsfit,g1[:-1],args=(rtotslist[0],satslist[0],ligslist[0]),bounds=lsbnds,ftol=1e-14,gtol=1e-14,xtol=1e-14,method='trf').x

array([   3.79527217,    5.44195648,    0.19624202,  170.09395461])

In [24]:
parmslist[0]

array([  4.60000000e+00,   5.30000000e+00,   3.40000000e-01,
         5.30000000e+02])

In [26]:
rtotslist[0]

array([ 0.001439,  0.002422,  0.003392,  0.002718,  0.01216 ,  0.02594 ])

In [27]:
def lsfitRTs(rtots,parms,data,ligs):
    datac = np.concatenate(data)
    calc = lb.models.wymfunc(parms,ligs,rtots)
    return (calc-datac)

In [28]:
lsfitRTs(rtotslist[0],parmslist[0],satslist[0],ligslist[0])

array([-0.01740799, -0.0206607 , -0.00976835, -0.01188224,  0.01045145,
       -0.01141733,  0.01320418,  0.06026973, -0.0270151 , -0.04181374,
       -0.00458064, -0.00304705,  0.02567486,  0.01349586, -0.01724201,
       -0.00175286, -0.00540371,  0.00432547,  0.00988487,  0.01566476,
       -0.0087546 , -0.01276428, -0.02144349,  0.00788486,  0.02452845,
       -0.00650372,  0.01042254, -0.01009578, -0.00958264,  0.02378302,
        0.03612123,  0.01638346, -0.0005144 ,  0.01484885,  0.01965084,
       -0.02493543, -0.02315243, -0.03071105,  0.01074289, -0.01619849,
        0.00173348, -0.00517401,  0.01857583,  0.02616626,  0.01311028,
        0.01056956,  0.00776938, -0.03126116, -0.00480073, -0.03001531,
        0.02937368, -0.00194262, -0.00599826,  0.03041262, -0.0116317 ,
        0.00239252,  0.01994538, -0.00769327, -0.00521469,  0.01212297,
        0.00850731, -0.01666098,  0.00871942,  0.01536787, -0.03444388,
       -0.00120259, -0.01421852, -0.00399386, -0.01403875, -0.00

In [29]:
rtguess = np.array([0.05,0.05,0.05,0.05,0.05,0.05])

In [41]:
rtbnds = (tuple(1e-5 for i in range(6)),tuple(1e-1 for i in range(6)))

In [100]:
sp.optimize.least_squares(lsfitRTs,rtguess,args=(parmslist[0],satslist[0],ligslist[0]),bounds=rtbnds,ftol=1e-14,gtol=1e-14,xtol=1e-14,method='dogbox').x

array([ 0.00143878,  0.00242228,  0.00339179,  0.0027181 ,  0.01216361,
        0.02594478])

In [32]:
rtotslist[0]

array([ 0.001439,  0.002422,  0.003392,  0.002718,  0.01216 ,  0.02594 ])

In [96]:
rtotswt08 = sp.optimize.least_squares(lsfitRTs,rtguess,args=(parmslist[0],satslist[0],ligslist[0]),bounds=rtbnds,ftol=1e-14,gtol=1e-14,xtol=1e-14,method='trf').x

In [97]:
rtotswt08

array([ 0.00143878,  0.00242228,  0.00339179,  0.0027181 ,  0.01216361,
        0.02594478])

In [36]:
prmguess = g1[:-1]

In [99]:
parmslist[0]

array([  4.60000000e+00,   5.30000000e+00,   3.40000000e-01,
         5.30000000e+02])

In [112]:
sp.optimize.least_squares(lsfit,prmguess,args=(rtotswt08,satslist[0],ligslist[0]),bounds=lsbnds,ftol=1e-15,gtol=1e-15,xtol=1e-15,method='dogbox').x

array([   3.79506217,    5.44228763,    0.19626185,  170.03729939])

In [43]:
parmslist[0]

array([  4.60000000e+00,   5.30000000e+00,   3.40000000e-01,
         5.30000000e+02])

In [44]:
nllbnds = ((0.0001,1000.),(0.0001,1000.),(0.0001,1000.),(0.001,10000.),(1e-6,0.5))

In [117]:
sp.optimize.minimize(nll,g1,args=(rtotswt08,satslist[0],ligslist[0]),method='L-BFGS-B',bounds=nllbnds,options={'ftol':1e-10,'gtol':1e-16,'eps':1e-14})

      fun: -229.17543917188368
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 612
      nit: 44
   status: 0
  success: True
        x: array([  3.83663854e+00,   6.01321880e+00,   1.56580119e-01,
         1.70000000e+02,   9.49039490e-02])

In [75]:
g1

array([  4.00000000e+00,   6.00000000e+00,   1.30000000e+01,
         1.70000000e+02,   1.00000000e-02])

In [68]:
sp.optimize.minimize(nllNW,g3,args=(rtotswt08,satslist[0],ligslist[0]),method='L-BFGS-B',bounds=nllbnds,options={'ftol':1e-16,'gtol':1e-16})

      fun: 0.012625145419737758
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.14491749e-07,  -1.63064007e-08,  -6.90246471e-07,
         1.21430643e-09,   0.00000000e+00])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 402
      nit: 44
   status: 0
  success: True
        x: array([  3.79509816e+00,   5.44224524e+00,   1.96265794e-01,
         1.70046686e+02,   1.00000000e-02])

In [67]:
parmslist[0]

array([  4.60000000e+00,   5.30000000e+00,   3.40000000e-01,
         5.30000000e+02])